## 1. Importación de las librerias necesarias

In [111]:
!pip install flask keras tensorflow scikit-learn
!pip install -U flask-cors

In [112]:
from flask import Flask, render_template, request
from flask_cors import CORS
import pickle

## 2. Importación del modelo y del DictVectorizer

In [14]:
model = pickle.load(open('modelo_coches_net.pkl', 'rb'))

2023-05-07 11:24:43.299480: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [17]:
with open('dict_vectorizer_coches_net.pck', 'rb') as f:
  dv = pickle.load(f)

## 3. Comprobación de que el modelo funciona

In [131]:
coche = {
    "fuelType": "Gasolina",
    "km": 55000.0,
    "make": "SMART",
    "model": "forfour",
    "province": "Alicante",
    "transmissionType": "COMMON.TRANSMISSION_AUTOMATIC",
    "year": 2016.0,
    "seller_type": "particular",
    "bodyType": "BODY_TYPE_SEDAN",
    "cubicCapacity": 999.0,
    "doors": 5.0,
    "hp": 71.0,
}
X_train = dv.transform(coche)
X_train

array([[0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 1.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        9.990e+02, 5.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        1.000e+00, 0.000e+00, 0.000e+00, 7.100e+01, 5.500e+04, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+0

In [128]:
coche = {
    "fuelType": "Gasolina",
    "km": 20000.0,
    "make": "AUDI",
    "model": "LEON",
    "province": "Almería",
    "transmissionType": "COMMON.TRANSMISSION_AUTOMATIC",
    "year": 1999.0,
    "seller_type": "particular",
    "bodyType": "BODY_TYPE_SEDAN",
    "cubicCapacity": 999.0,
    "doors": 5.0,
    "hp": 71.0,
}
X_train = dv.transform(coche)
X_train


array([[0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 1.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        9.990e+02, 5.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        1.000e+00, 0.000e+00, 0.000e+00, 7.100e+01, 2.000e+04, 0.000e+00,
        0.000e+00, 1.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+0

In [132]:
X_train.shape[1]

542

In [133]:
precio = model.predict(X_train)
precio

1/1 [==============================] - 0s 19ms/step


array([[13209.061]], dtype=float32)

## 4. Aplicación Flask

In [135]:
app = Flask(__name__)
CORS(app)

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/predecir', methods=['POST'])
def predecir():
    coche = request.get_json()
    print(coche)
    if coche is not None:
        print('data:', [dict(coche)])
        X_train = dv.transform([dict(coche)])
        precio = model.predict(X_train)
        print('precio', precio)
        return {'precio': round(float(precio[0]), 2)}
    else:
        return 'No valido!'

app.run()

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [08/May/2023 10:24:35] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/May/2023 10:26:09] "OPTIONS /predecir HTTP/1.1" 200 -


{'brand': 'AUDI', 'model': 'LEON', 'km': '20000', 'year': '1999', 'fueltype': 'Gasolina', 'cubiccapacity': '999', 'province': 'Almería', 'bodytype': 'BODY_TYPE_SEDAN', 'seller_type': 'particular', 'hp': '71', 'transmissiontype': 'AUTOMATIC', 'doors': '5', 'color': 'BLUE'}
data: [{'brand': 'AUDI', 'model': 'LEON', 'km': '20000', 'year': '1999', 'fueltype': 'Gasolina', 'cubiccapacity': '999', 'province': 'Almería', 'bodytype': 'BODY_TYPE_SEDAN', 'seller_type': 'particular', 'hp': '71', 'transmissiontype': 'AUTOMATIC', 'doors': '5', 'color': 'BLUE'}]
1/1 [==============================] - 0s 21ms/step


127.0.0.1 - - [08/May/2023 10:26:09] "POST /predecir HTTP/1.1" 200 -


precio [[-29.274908]]
